In [1]:
%matplotlib widget

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import sys

from PfyMU.gait.train_classifier.core import load_datasets

In [3]:
gait_sets_path = Path('/Users/adamol/Documents/Datasets/gait/processed')

datasets = [
    gait_sets_path / 'bluesky2',
    gait_sets_path / 'daliac',
    gait_sets_path / 'ltmm',
    gait_sets_path / 'usc-had'
]

X, subjects, Y = load_datasets(datasets, goal_fs=50.0, acc_mag=True, window_length=3.0, window_step=0.5)

## Dataset class summary

In [4]:
print('Total samples (3.0s windows): ', Y.size)
print('Total walking samples: ', Y.sum())
print('Total non-walking samples: ', Y.size - Y.sum(), '\n')
print(f'% walking samples: {Y.sum() / Y.size * 100:.2f}')

Total samples (3.0s windows):  82308
Total walking samples:  15345
Total non-walking samples:  66963 

% walking samples: 18.64


## Feature Generation

In [5]:
from PfyMU.features import *

In [6]:
FB = Bank(window_length=None, window_step=None)

# add features
FB + Mean()
FB + MeanCrossRate()
FB + StdDev()
FB + Skewness()
FB + Kurtosis()
FB + Range()
FB + IQR()
FB + RMS()
FB + Autocorrelation(lag=1, normalize=True)
FB + LinearSlope()
FB + SignalEntropy()
FB + SampleEntropy(m=4, r=1.0)
FB + PermutationEntropy(order=3, delay=1, normalize=True)
FB + ComplexityInvariantDistance(normalize=True)
FB + RangeCountPercentage(range_min=0, range_max=1.0)
FB + RatioBeyondRSigma(r=2.0)
FB + JerkMetric(normalize=True)
FB + DimensionlessJerk(log=True, signal_type='acceleration')
FB + SPARC()
FB + DominantFrequency(low_cutoff=0.25, high_cutoff=5.0)
FB + DominantFrequencyValue(low_cutoff=0.25, high_cutoff=5.0)
FB + PowerSpectralSum(low_cutoff=0.25, high_cutoff=5.0)
FB + SpectralFlatness(low_cutoff=0.25, high_cutoff=5.0)
FB + SpectralEntropy(low_cutoff=0.25, high_cutoff=5.0)
FB + DetailPower(wavelet='coif4', freq_band=[1.0, 3.0])
FB + DetailPowerRatio(wavelet='coif4', freq_band=[1.0, 3.0])

In [7]:
X_feat = FB.compute(X, fs=50.0, windowed=True)

/Users/adamol/opt/miniconda3/envs/pfymu/lib/python3.8/site-packages/pywt/_multilevel.py:43: UserWarning: Level value of 6 is too high: all coefficients will experience boundary effects.
  warnings.warn(


## Feature Exploration

In [8]:
feats = pd.DataFrame(data=X_feat, columns=[i.parent._name for i in FB._feat_list])
feats['Labels'] = Y

feat_names = [i.parent._name for i in FB._feat_list]

### Feature distributions

In [9]:
plt.close('all')

f, ax = plt.subplots(nrows=7, ncols=4, figsize=(10, 10), sharex=True)

j, k = 0, 0
for i, ft in enumerate([i.parent._name for i in FB._feat_list]):
    if k > 3:
        j += 1
        k  = 0
    sns.violinplot(x='Labels', y=ft, data=feats, ax=ax[j, k])
    ax[j, k].set_ylabel(None)
    ax[j, k].set_title(ft)
    ax[j, k].set_xlabel(None)
    
    k += 1

for k in range(4):
    ax[-1, k].set_xticklabels(['NG', 'G'])

f.tight_layout()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

### Predictive Power Score

In [10]:
import ppscore

In [11]:
df_predictors = ppscore.predictors(feats, 'Labels', output='df')
plt.figure()
ax = sns.barplot(data=df_predictors, x="x", y="ppscore")
ax.set_xticklabels(ax.get_xticklabels(), rotation=45, ha="right")
plt.tight_layout()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [12]:
pps_matrix = ppscore.matrix(feats)

In [19]:
plt.figure(figsize=(15, 10))
sns.heatmap(pps_matrix, vmin=0, vmax=1, cmap="Blues", linewidths=0.5, annot=True)
plt.tight_layout()
# plt.savefig('PPScore_matrix.pdf')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [14]:
from sklearn.decomposition import PCA

In [15]:
pca = PCA(n_components=5)
x_pca = pca.fit_transform(feats)

In [16]:
f, ax = plt.subplots(nrows=5, ncols=5, figsize=(10, 6))
for i in range(5):
    for j in range(5):
        ax[i, j].loglog(x_pca[~Y.astype(bool), i], x_pca[~Y.astype(bool), j], '.', color='C0', label='Non-gait')
        ax[i, j].loglog(x_pca[Y.astype(bool), i], x_pca[Y.astype(bool), j], '.', color='C1', label='Gait')

for i in range(5):
    ax[i, 0].set_ylabel(f'PC {i}')
for j in range(5):
    ax[-1, j].set_xlabel(f'PC {j}')

f.tight_layout()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [17]:
from separation_stats import db_2class, corr_select, cohen_d, ttest_select, mwu_select, auc_score

In [18]:
sep_df = pd.DataFrame()
sep_df['Feature'] = feat_names

sep_df['DBI'], db_rank = db_2class(X_feat, Y)
sep_df['Correlation r'] = corr_select(X_feat, Y)
sep_df['T-test p'], sep_df['Cohens d'] = ttest_select(X_feat, Y)
sep_df['MWU-test p'], _ = mwu_select(X_feat, Y)
sep_df['AUC'] = auc_score(X_feat, Y)

sep_df['Cohens d'] = sep_df['Cohens d'].abs()
sep_df['AUC'] = sep_df['AUC'].apply(lambda i: 1 - i if i < 0.5 else i)

In [20]:
sep_df.sort_values('DBI')

,Feature,DBI,Correlation r,T-test p,Cohens d,MWU-test p,AUC
21,PowerSpectralSum,0.576321,-0.661210,0.000000e+00,2.263070,0.000000,0.921700
20,DominantFrequencyValue,0.580382,-0.580590,0.000000e+00,1.830949,0.000000,0.897360
17,DimensionlessJerk,0.608611,-0.570703,0.000000e+00,1.784512,0.000000,0.900161
12,PermutationEntropy,0.734115,-0.612314,0.000000e+00,1.988589,0.000000,0.916372
19,DominantFrequency,0.753355,0.397789,0.000000e+00,1.113252,0.000000,0.745626
1,MeanCrossRate,0.768326,-0.528784,0.000000e+00,1.599672,0.000000,0.899688
23,SpectralEntropy,0.795675,0.560333,0.000000e+00,1.737047,0.000000,0.881460
13,ComplexityInvariantDistance,0.940949,-0.488800,0.000000e+00,1.438644,0.000000,0.874255
14,RangeCountPercentage,0.987232,0.368835,0.000000e+00,1.018874,0.000000,0.786030
22,SpectralFlatness,0.993416,0.439278,0.000000e+00,1.255535,0.000000,0.803157


In [21]:
sep_df.sort_values('Cohens d', ascending=False)

,Feature,DBI,Correlation r,T-test p,Cohens d,MWU-test p,AUC
21,PowerSpectralSum,0.576321,-0.661210,0.000000e+00,2.263070,0.000000,0.921700
12,PermutationEntropy,0.734115,-0.612314,0.000000e+00,1.988589,0.000000,0.916372
20,DominantFrequencyValue,0.580382,-0.580590,0.000000e+00,1.830949,0.000000,0.897360
17,DimensionlessJerk,0.608611,-0.570703,0.000000e+00,1.784512,0.000000,0.900161
23,SpectralEntropy,0.795675,0.560333,0.000000e+00,1.737047,0.000000,0.881460
1,MeanCrossRate,0.768326,-0.528784,0.000000e+00,1.599672,0.000000,0.899688
13,ComplexityInvariantDistance,0.940949,-0.488800,0.000000e+00,1.438644,0.000000,0.874255
8,Autocorrelation,1.100473,0.448261,0.000000e+00,1.287588,0.000000,0.874449
22,SpectralFlatness,0.993416,0.439278,0.000000e+00,1.255535,0.000000,0.803157
19,DominantFrequency,0.753355,0.397789,0.000000e+00,1.113252,0.000000,0.745626


In [22]:
sep_df.sort_values('AUC', ascending=False)

,Feature,DBI,Correlation r,T-test p,Cohens d,MWU-test p,AUC
21,PowerSpectralSum,0.576321,-0.661210,0.000000e+00,2.263070,0.000000,0.921700
12,PermutationEntropy,0.734115,-0.612314,0.000000e+00,1.988589,0.000000,0.916372
6,IQR,1.176929,0.317492,0.000000e+00,0.859687,0.000000,0.903084
25,DetailPowerRatio,2.353642,0.201746,0.000000e+00,0.528888,0.000000,0.901499
16,JerkMetric,1.269824,0.308233,0.000000e+00,0.831941,0.000000,0.900161
17,DimensionlessJerk,0.608611,-0.570703,0.000000e+00,1.784512,0.000000,0.900161
1,MeanCrossRate,0.768326,-0.528784,0.000000e+00,1.599672,0.000000,0.899688
2,StdDev,1.211330,0.266670,0.000000e+00,0.710441,0.000000,0.898626
7,RMS,1.211330,0.266670,0.000000e+00,0.710441,0.000000,0.898626
24,DetailPower,4.942013,0.045775,1.963653e-39,0.117657,0.000000,0.898490


### Pair plots (Top Features)

In [26]:
top_feats = ['PowerSpectralSum', 'PermutationEntropy', 'IQR', 'DominantFrequency', 'DimensionlessJerk', 'DetailPowerRatio']

sns.pairplot(feats, hue='Labels', vars=top_feats)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

/Users/adamol/opt/miniconda3/envs/pfymu/lib/python3.8/site-packages/seaborn/distributions.py:369: UserWarning: Default bandwidth for data is 0; skipping density estimation.
  warnings.warn(msg, UserWarning)
